In [196]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.keys import Keys
from datetime import datetime, timedelta
import requests
import time



__author__ = 'aituarov'


DB_DATASET_DIR = 'C:\\DEV\\output\\'
chrome_options = Options()

chrome_options.add_experimental_option( "prefs",    

{
  "download.default_directory": DB_DATASET_DIR,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True

})

driver = webdriver.Chrome(options = chrome_options)

end_date = datetime.today().date() + timedelta(days=-1)
start_date = end_date + timedelta(days=-1)

end_date = datetime.strptime("2019-05-30", '%Y-%m-%d').date()
start_date = datetime.strptime("2019-05-25", '%Y-%m-%d').date()



# def retrieve_unavailability():
#     print(' - Unavailability - ')
    
#     url = 'http://www2018.terna.it/it-it/sistemaelettrico/transparencyreport/generation/exanteinformationonplannedoutagesofgenerationunits.aspx'
# #     outdir = DB_DATASET_DIR + 'Terna/unavailability/'
#     outdir = DB_DATASET_DIR
    
#     driver = webdriver.Chrome(options = chrome_options)
#     driver.get(url)
#     time.sleep(2)
    
    
#     yr_select = driver.find_element_by_id('dnn_ctr5966_TernaViewDocumentView_cbAnno_Input')
#     yr_select.click()
#     yr_select.send_keys(Keys.UP)
#     yr_select.send_keys(Keys.UP)
#     yr_select.send_keys(Keys.UP)
#     yr_select.send_keys(Keys.UP)
#     driver.find_element_by_id('dnn_ctr5966_TernaViewDocumentView_cmdFilter').click()
#     time.sleep(3)
    
    
#     size_select = driver.find_element_by_id('dnn_ctr5966_TernaViewDocumentView_grdDocument_ctl00_ctl03_ctl01_PageSizeComboBox_Input')
#     size_select.click()
#     size_select.send_keys(Keys.DOWN)
#     size_select.send_keys(Keys.DOWN)
#     table_size_50 = driver.find_element_by_xpath('//div[@id="dnn_ctr5966_TernaViewDocumentView_grdDocument_ctl00_ctl03_ctl01_PageSizeComboBox_DropDown"]//li[contains(., "50")]')
#     table_size_50.click()

#     time.sleep(3)
    
    
#     yr_list = []
    
#     first_page_num = 1
    
#     last_page_num = driver.find_element_by_xpath('//*[@class="rgWrap rgInfoPart"]').text.split()[3]

#     for current_page in range(first_page_num, int(last_page_num)+1):
#         pages = driver.find_elements_by_xpath('//*[@class="rgWrap rgNumPart"]/a')
        
#         tab = driver.find_element_by_xpath('//*[@id="dnn_ctr5966_TernaViewDocumentView_grdDocument_ctl00"]/tbody')

#         rows = tab.find_elements_by_tag_name('tr')
        
#         for tr in rows:
            
#             if tr.find_elements_by_tag_name('td')[1].text != '':

#                 pubdte = datetime.strptime(tr.find_elements_by_tag_name('td')[1].text,  '%d/%m/%Y')

#                 if pubdte.year not in yr_list:  # only last file for each available year

#                     try:
#                         txt = tr.find_elements_by_tag_name('td')[0].text

#                         fileurl = tr.find_elements_by_tag_name('a')[0].get_attribute('href')

#                         resp = requests.get(fileurl)

#                         outfile = format_name(txt).replace(' ', '_') + pubdte.strftime('_%Y_%m_%d') + ".xls"

#                         print(outfile)

#                         with open(outdir + outfile, 'wb') as output:

#                             output.write(resp.content)

#                             output.close()

#                         print(' downloaded')

#                         yr_list.append(pubdte.year)

#                         time.sleep(1)

#                     except:
#                         print("Error")

#         print()
#         if (current_page + 1) <= int(last_page_num):
#             pos = yr_select.location_once_scrolled_into_view
#             driver.execute_script('window.scrollTo(0,' + str(pos['y']) + ')')  # scroll down to call function
#             go_next_page_table(current_page+1, pages)
#             time.sleep(5)
    
#     print("All files downloaded, closing browser...")
#     driver.quit()
            
            
            
# def go_next_page_table(next_page_num, pages_element):
#     for el in pages_element:
#         if el.text == str(next_page_num):
#             next_page = el
#     if len(pages_element) == next_page_num:
#         next_page = pages_element[-1]
#     print("Going to " + str(next_page_num) + " page")
#     next_page.click()
    
    
    
def retrieve_capacita(start_date, end_date):
    outdir = DB_DATASET_DIR
    
    url = 'https://www.terna.it/it/sistema-elettrico/dispacciamento/capacita-produttiva-disponibile'
    driver.get(url)
    request_date = end_date
    
    driver.find_element_by_xpath('//div[@class="grid-filters"]//option[@value="' + str(request_date.month-1) +'"]').click()
    driver.find_element_by_xpath('//div[@class="grid-filters"]//option[@value="' + str(request_date.year) +'"]').click()
    driver.find_element_by_xpath('//div[@class="search-button-wrapper"]').click()
    time.sleep(3)
    
    rows = driver.find_elements_by_xpath('//div[@ref="eCenterContainer"]/div')
    
    while request_date >= start_date:
        cnt_files = 3
        cnt_files = find_download_files(rows, request_date, outdir, cnt_files)
            
        if cnt_files == 0:
            request_date += timedelta(days=-1)
        else:
            driver.find_element_by_xpath('//div[@class="pagination-container"]//a[@aria-label="Next"]').click()
            time.sleep(3)
            rows = driver.find_elements_by_xpath('//div[@ref="eCenterContainer"]/div')
            cnt_files = find_download_files(rows, request_date, outdir, cnt_files)
            if cnt_files == 0:
                request_date += timedelta(days=-1)
            else:
                print("Error, can't find all files on " + str(request_date))
                        
    print("All files downloaded")
            
    
def find_download_files(rows, request_date, outdir, cnt_files):
    for row in rows:
        pub_dte = row.find_element_by_xpath('.//div[@col-id="date"]').text
        file_name = row.find_element_by_xpath('.//div[@col-id="titolo_documento"]').text

        if datetime.strptime(pub_dte, '%d/%m/%Y').date() == request_date:
            fileurl = row.find_element_by_tag_name('a').get_attribute('href')
            resp = requests.get(fileurl)

            file_name = format_name(file_name) + '_' + str(request_date).replace('-', '') + ".xlsx"
            print(file_name)
            with open(outdir + file_name, 'wb') as output:
                output.write(resp.content)
                output.close()

            print(' downloaded')
            cnt_files-=1
    return cnt_files



def format_name(str):

    dic ={
            u'\xe0': 'a', u'\xe1': 'a', u'\xe2': 'a', u'\xe3': 'a', u'\xe4': 'a', u'\xe5': 'a', u'\xe6': 'a', 
        
            u'\xe7': 'c',
        
            u'\xe8': 'e', u'\xe9': 'e', u'\xea': 'e', u'\xeb': 'e',
        
            u'\xf1': 'n',
        
            u'\xf2': 'o', u'\xf3': 'o', u'\xf4': 'o', u'\xf6': 'o',
        
            u'\xed': 'i', u'\xee': 'i',
        
            u'\xfa': 'u', u'\xfb': 'u', u'\xfc': 'u',
        
            u'\xf0': 'd',
        
            u'\xfb': 'th',
        
            u'\xfd': 'y',
        
            '\\': '-', '/': '-','#': '-',
        
            "'": '', '?': '', '\x92': '', '=': '', '*': '', ',': '',
        }
    
    for d in dic:
        str = str.lower().replace(d,dic[d])

    return str

                
                
    
    
def main():

#     retrieve_unavailability()

    retrieve_capacita(start_date, end_date)



if __name__ == '__main__':

    main()




capacita disponibile dettaglio_20190530.xlsx
 downloaded
capacita disponibile aggregata_20190530.xlsx
 downloaded
capacita disponibile attesa_20190530.xlsx
 downloaded
capacita disponibile attesa_20190529.xlsx
 downloaded
capacita disponibile aggregata_20190529.xlsx
 downloaded
capacita disponibile dettaglio_20190529.xlsx
 downloaded
capacita disponibile dettaglio_20190528.xlsx
 downloaded
capacita disponibile aggregata_20190528.xlsx
 downloaded
capacita disponibile attesa_20190528.xlsx
 downloaded
capacita disponibile dettaglio_20190527.xlsx
 downloaded
capacita disponibile aggregata_20190527.xlsx
 downloaded
capacita disponibile attesa_20190527.xlsx
 downloaded
capacita disponibile dettaglio_20190526.xlsx
 downloaded
capacita disponibile aggregata_20190526.xlsx
 downloaded
capacita disponibile attesa_20190526.xlsx
 downloaded
capacita disponibile dettaglio_20190525.xlsx
 downloaded
capacita disponibile aggregata_20190525.xlsx
 downloaded
capacita disponibile attesa_20190525.xlsx
 dow

In [199]:
request_date = datetime.today().date()
# request_date += timedelta(days=-1)
request_date.day

1

In [215]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
import requests
import time



__author__ = 'aituarov'


DB_DATASET_DIR = 'C:\\DEV\\output\\'
chrome_options = Options()

chrome_options.add_experimental_option( "prefs",    

{
  "download.default_directory": DB_DATASET_DIR,
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True

})

driver = webdriver.Chrome(options = chrome_options)

end_date = datetime.today().date() + timedelta(days=-1)
start_date = end_date + timedelta(days=-5)

start_date = datetime.strptime("2020-06-13", '%Y-%m-%d').date()
end_date = datetime.strptime("2020-06-23", '%Y-%m-%d').date()

    
    
def retrieve_capacita(start_date, end_date):
    outdir = DB_DATASET_DIR
    
    url = 'https://www.terna.it/it/sistema-elettrico/dispacciamento/capacita-produttiva-disponibile'
    driver.get(url)
    request_date = end_date
    
    select_month = driver.find_element_by_xpath('//div[@class="grid-filters"]//option[@value="' + str(request_date.month-1) +'"]')
    select_month.click()
    driver.find_element_by_xpath('//div[@class="grid-filters"]//option[@value="' + str(request_date.year) +'"]').click()
    driver.find_element_by_xpath('//div[@class="search-button-wrapper"]').click()
    time.sleep(3)
    
    rows = driver.find_elements_by_xpath('//div[@ref="eCenterContainer"]/div')
    table_page = 1
 
    last_pub_dte = rows[0].find_element_by_xpath('.//div[@col-id="date"]').text
    if request_date > datetime.strptime(last_pub_dte, '%d/%m/%Y').date():
        print("Last published file's date " + last_pub_dte)
        print("Please check your end date")
        
    else:
        while request_date >= start_date:      
            cnt_files = 3
            cnt_files = find_download_files(rows, request_date, outdir, cnt_files)

            if cnt_files == 0:
                request_date += timedelta(days=-1)
            else:
                print("Going to " + str(table_page + 1) + " page to find files on " + str(request_date))
                driver.find_element_by_xpath('//div[@class="pagination-container"]//a[@aria-label="Next"]').click()
                table_page+=1
                time.sleep(1)
                rows = driver.find_elements_by_xpath('//div[@ref="eCenterContainer"]/div')
                cnt_files = find_download_files(rows, request_date, outdir, cnt_files)
                if cnt_files == 0:
                    request_date += timedelta(days=-1)
                elif cnt_files != 0 and table_page > 10:
                    pos = select_month.location_once_scrolled_into_view
                    driver.execute_script('window.scrollTo(0,' + str(pos['y']) + ')')  # scroll up to press button
                    driver.find_element_by_xpath('//div[@class="grid-filters"]//option[@value="' + str(request_date.month-2) +'"]').click()
                    driver.find_element_by_xpath('//div[@class="search-button-wrapper"]').click()
                    time.sleep(3)

                    rows = driver.find_elements_by_xpath('//div[@ref="eCenterContainer"]/div')
                    table_page = 1
                    cnt_files = find_download_files(rows, request_date, outdir, cnt_files)
                    if cnt_files == 0:
                        request_date += timedelta(days=-1)

        print("All files downloaded")
            
    
def find_download_files(rows, request_date, outdir, cnt_files):
    for row in rows:
        pub_dte = row.find_element_by_xpath('.//div[@col-id="date"]').text
        file_name = row.find_element_by_xpath('.//div[@col-id="titolo_documento"]').text

        if datetime.strptime(pub_dte, '%d/%m/%Y').date() == request_date:
            fileurl = row.find_element_by_tag_name('a').get_attribute('href')
            resp = requests.get(fileurl)

            file_name = format_name(file_name) + '_' + str(request_date).replace('-', '') + ".xlsx"
            print(file_name)
            with open(outdir + file_name, 'wb') as output:
                output.write(resp.content)
                output.close()

            print(' downloaded')
            cnt_files-=1
    return cnt_files



def format_name(str):

    dic ={
            u'\xe0': 'a', u'\xe1': 'a', u'\xe2': 'a', u'\xe3': 'a', u'\xe4': 'a', u'\xe5': 'a', u'\xe6': 'a', 
        
            u'\xe7': 'c',
        
            u'\xe8': 'e', u'\xe9': 'e', u'\xea': 'e', u'\xeb': 'e',
        
            u'\xf1': 'n',
        
            u'\xf2': 'o', u'\xf3': 'o', u'\xf4': 'o', u'\xf6': 'o',
        
            u'\xed': 'i', u'\xee': 'i',
        
            u'\xfa': 'u', u'\xfb': 'u', u'\xfc': 'u',
        
            u'\xf0': 'd',
        
            u'\xfb': 'th',
        
            u'\xfd': 'y',
        
            '\\': '-', '/': '-','#': '-',
        
            "'": '', '?': '', '\x92': '', '=': '', '*': '', ',': '',
        }
    
    for d in dic:
        str = str.lower().replace(d,dic[d])

    return str

                
                
    
    
def main():
    
    retrieve_capacita(start_date, end_date)



if __name__ == '__main__':

    main()




capacita disponibile aggregata_20200623.xlsx
 downloaded
capacita disponibile attesa_20200623.xlsx
 downloaded
capacita disponibile dettaglio_20200623.xlsx
 downloaded
capacita disponibile dettaglio_20200622.xlsx
 downloaded
capacita disponibile aggregata_20200622.xlsx
 downloaded
capacita disponibile attesa_20200622.xlsx
 downloaded
capacita disponibile dettaglio_20200621.xlsx
 downloaded
capacita disponibile aggregata_20200621.xlsx
 downloaded
capacita disponibile attesa_20200621.xlsx
 downloaded
capacita disponibile dettaglio_20200620.xlsx
 downloaded
Going to 2 page to find files on 2020-06-20
capacita disponibile aggregata_20200620.xlsx
 downloaded
capacita disponibile attesa_20200620.xlsx
 downloaded
capacita disponibile aggregata_20200619.xlsx
 downloaded
capacita disponibile dettaglio_20200619.xlsx
 downloaded
capacita disponibile attesa_20200619.xlsx
 downloaded
capacita disponibile aggregata_20200618.xlsx
 downloaded
capacita disponibile dettaglio_20200618.xlsx
 downloaded
ca